In [ ]:
# Tiny Shakespeare Dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-21 13:48:08--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.005s  

2023-04-21 13:48:10 (197 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
import torch

In [2]:
cuda = torch.cuda.is_available()
print(cuda, torch.cuda.get_device_name())

True NVIDIA GeForce GTX 1650 SUPER


In [3]:
# Reading the file
file = open('input.txt', 'r', encoding='utf-8')
text = file.read()
# text = text.lower()

In [4]:
print("# Characters: ", len(text))

# Characters:  1115394


In [5]:
# First 1000 characters
# print(text[:1000])

In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# Map creation
stoi = {}
itos = {}
for idx, char in enumerate(chars):
  stoi[char] = idx
  itos[idx] = char

# print(stoi)
# print(itos)

In [8]:
def encode(text: str):
  output = list(range(len(text)))
  for idx, char in enumerate(text):
    output[idx] = stoi[char]

  return output

def decode(arr: list[int]):
  output = list(range(len(arr)))
  for idx in range(len(arr)):
    output[idx] = itos[arr[idx]]

  return "".join(output)

In [9]:
# Encoding the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
# print(data[:1000]) # First 1000 characters in index format as per the vocab

torch.Size([1115394]) torch.int64


In [10]:
# Split up the data into train and validation sets
split = int(0.9 * len(data)) # First 90% will be training data, rest validation data
train_data = data[:split]
val_data = data[split:]
context = 8

In [11]:
x = train_data[:context]
y = train_data[1:context+1]
for ptr in range(context):
    context = x[:ptr+1] # Input till 'ptr+1' elements
    target = y[ptr] # Output is y[pt] since y is already starting from x+1
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [32]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
context = 8 # what is the maximum context window for predictions?
# Generate a small batch of data of inputs x and targets y
def get_batch(split):
    data = train_data if split == 'train' else val_data

    # ix is a list of random offsets. Length of ix list is equal to the batch_size. It is essentially used to sample "batch_size" number of rows from the data
    ix = torch.randint(len(data) - context, (batch_size,))
    # print(f"ix is: {ix}")
    x = list(range(len(ix)))
    y = list(range(len(ix)))
    # print(f"Data is: {data[0]}")
    for idx, offset in enumerate(ix):
      x[idx] = data[offset : offset+context]
      y[idx] = data[offset+1 : offset+context+1]

    # print(len(x), len(y))
    x = torch.stack(x)
    y = torch.stack(y)

    return x, y

xb, yb = get_batch('train')
print('Inputs:')
print(xb.shape)
print(xb)
print('Targets:')
print(yb.shape)
print(yb)

# print('----')

for batch in range(batch_size): # batch dimension
  print("----")
  for pt in range(context):
      context = xb[batch][:pt+1]
      target = yb[batch][pt]
      print(f"When input is {context.tolist()} the target: {target.item()}")

Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When input is [24] the target: 43
When input is [24, 43] the target: 58
When input is [24, 43, 58] the target: 5
When input is [24, 43, 58, 5] the target: 57
When input is [24, 43, 58, 5, 57] the target: 1
When input is [24, 43, 58, 5, 57, 1] the target: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
----
When input is [44] the target: 53
When input is [44, 53] the target: 56
When input is [44, 53, 56] the target: 1
When input is [44, 53, 56, 1] the target: 58
When input is [44, 53, 56, 1, 58] the target: 46
When input is [4

In [33]:
print(xb) # Our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [34]:
import torch.nn as nn
from torch.nn import functional as functional
torch.manual_seed(1337)

In [48]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        
        # Dimensions for this embedding layer is equal to the vocab size
        self.dimensionality = vocab_size

        # Embedding layer is essentially a look up table
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=self.dimensionality)
        # Embedding table is (vocab size, dimensionality)

    def forward(self, idx, targets=None):

        # If idx is a scalar, logit is a 'dimension' long (60) tensor
        # If idx is a vector, logit is a tensor of shape(len(idx), dimensionality) 
        # If idx is a tensor of shape(m,n), logit is a tensor of shape(m,n,dimensionality)
        # Essentially when idx is a tensor, the same tensor is returned, but instead of indices, each tensor value is a word vector 
        # idx is (batch_size, context_window)
        # targets is also (batch_size, context_window)
        logits = self.embedding(idx) # logit is (batch_size, context_window, word vector dimensionality)
        if targets is None:
            loss = None
        else:
            # Cross entropy Function: 
            # If logits is an vector, target is a single number
            # If logits is a tensor of shape (count, probabilities), targets is a vector of length "count" 

            # Hence we are resizing the logits and targets 
            # Here we are clubbing all the batches together in both the logits and the targets
            # For each singular target, we have a probability distribution over "dimensionality" number of elements
            # Since "dimensionality" is equal to "vocab_size", the probability is over all the vocab elements
            # Hence the cross entropy calculate loss between the target element and the output element from the probabilities 
            targets = targets.view(batch_size * context)
            logits = logits.view(batch_size * context, self.dimensionality)
            loss = functional.cross_entropy(logits, targets)      
        return logits, loss
        
    
    def generate(self, idx, max_new_tokens=1):
        # idx is (batch_size, context_window) Matrix of indices in the current context

        for _ in range(max_new_tokens):

            # Get the predictions
            logits, loss = self.forward(idx)
            # Focus only on the last  step which contains the output considering the entire context window

            # logits are (batch_size, dimensionality) which is essentially the output vector for each batch
            logits = logits[:, -1, :] 
            # Apply softmax to get probabilities
            probs = functional.softmax(logits, dim=1)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence

            # appended along the context_window hence the context keeps building up
            idx = torch.cat((idx, idx_next), dim=1) # (batch_size, context_window + 1)
        return idx

model = BigramLanguageModel(vocab_size)
logits, loss = model.forward(xb, yb)
# print(logits.shape, loss)
out = model.generate(xb, 1)
# print(out.shape)

start_idx = torch.zeros((1,1), dtype=torch.long)
# print(decode(bigram.generate(idx=start_idx, max_new_tokens=1)[0].tolist()))


In [49]:
# Create a PyTorch optimizer for the model parameters
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [52]:
batch_size = 32

# Training Epochs
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
        # Backward Pass
    for param in model.parameters():
        param.grad = None

    loss.backward()

    # Update
    for param in model.parameters():
        param.data -= 0.1 * param.grad

    print(loss.item())


2.7127552032470703
2.733153820037842
2.779812812805176
2.739114284515381
2.698676586151123
2.9273159503936768
2.784470319747925
2.65157413482666
2.711611747741699
2.6762514114379883
2.79891037940979
2.6301321983337402
2.710057020187378
2.585120439529419
2.858333110809326
2.902344226837158
2.847288131713867
2.7876136302948
2.7782652378082275
2.840991497039795
2.810853958129883
2.629575490951538
2.7360682487487793
2.97629451751709
2.7314350605010986
2.7220757007598877
2.686943292617798
2.7773640155792236
2.8247194290161133
2.7301559448242188
2.6515603065490723
2.713615655899048
2.6417312622070312
2.711982488632202
2.6647322177886963
3.052882194519043
2.4448604583740234
2.604144334793091
2.6694512367248535
2.7698609828948975
2.8208940029144287
2.7981808185577393
2.721022844314575
2.7781476974487305
2.7298505306243896
2.71217679977417
2.612802028656006
2.641143321990967
2.884514093399048
2.7548375129699707
2.6091697216033936
2.9109976291656494
2.8128018379211426
2.787048578262329
2.8737654

In [63]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


S:
Ancea fearl tkThePKQq?n!pD, ceeaicq;PWe h f t co;

IiJthie y ereJrperd fnd t oncAndsshiqmeVPYXl be llecers olly ke wes oce n R--MX?Gm.

O!Zd,
Lle!Xfe
Gbelwicestsce ithend l blouerijxXjve oulour'ds te h I's il$PJzCKso or h, hal w-dbenG, yTThVM:
F'nenthepeth couveJgTheY$Nr the GYEVPTEbrsa;
Irq, b-OjmoXjJPS:
LXV.
Gher hal dvanoter hiKoseasu by to ampsu lis
$Houls, IfRS:
ASR my buRkbor sheeLF
OjS, atalowSTXU so,
TwXvem?
WQpe himeet or ikLufo ithe bYKd,
BUCKM-e jk her,Xlowilon, te, me WbbeHYbz
US:


## The Mathematical Trick in Self-Attention

In [69]:
# Example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)

# Tril returns the left lower triangle of the tensor
# torch.ones(3, 3) = [
#   [1, 1, 1],
#   [1, 1, 1],
#   [1, 1, 1],
# ]

# torch.tril(torch.ones(3, 3)) = [
#   [1, 0, 0],
#   [1, 1, 0],
#   [1, 1, 1],
# ]

ones = torch.ones(3, 3)
print("Ones = ")
print(ones)
print()

a = torch.tril(torch.ones(3, 3))
print("Lower left triangular tensor 'tril' = ")
print(a)
print()

a = a / torch.sum(a, 1, keepdim=True)
print("Averaged tril 'a' = ")
print(a)
print()

b = torch.randint(low=0, high=10, size=(3,2)).float()
print("b = ")
print(b)
print()

c = a @ b

print('a @ b => c = ')
print(c)

Ones = 
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

Lower left triangular tensor 'tril' = 
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

Averaged tril 'a' = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

b = 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

a @ b => c = 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [71]:
# Consider the following example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time(context), channels(dimensionality)
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [76]:
# V1: Using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow = wei @ x # (B, T, T) @ (B, T, C) => (B, T, C)
print(xbow)


tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


In [77]:
# V2: Use Softmax
tril = torch.tril(torch.ones(T, T))
print("Tril = ")
print(tril)
print()

wei = torch.zeros((T,T))
print("Wei before = ")
print(wei)
print()

# Masked fill here will convert all the 0s of tril to -inf and superimpose them onto wei
wei = wei.masked_fill(tril == 0, float('-inf'))
print("Wei after = ")
print(wei)
print()

wei = functional.softmax(wei, dim=-1)
xbow = wei @ x
print(xbow)


Tril = 
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

Wei before = 
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

Wei after = 
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 

In [79]:
# V3: Self-Attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# We can think of "wei" matrix as the affinity matrix of input tokens, taken across different context lengths

# Earlier "wei" was same for all the tokens
# In Self Attention, the "wei" is data dependent

# Single Head Self-Attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)   # What the current token contains
query = nn.Linear(C, head_size, bias=False) # What the current token is looking for
value = nn.Linear(C, head_size, bias=False) # Additional value addition to input "x"
key_for_x =  key(x)   # (B, T, 16)
query_for_x = query(x) # (B, T, 16)
wei_for_x=  query_for_x @ key_for_x.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) => (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei_for_x = wei_for_x.masked_fill(tril == 0, float('-inf'))
wei_for_x = functional.softmax(wei, dim=-1)

value_for_x = value(x)
out = wei_for_x @ value_for_x

out.shape

torch.Size([4, 8, 16])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [81]:
num_emb = 32

# Single Self Attention Head
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(num_emb, head_size, bias=False)
        self.query = nn.Linear(num_emb, head_size, bias=False)
        self.value = nn.Linear(num_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(context, context)))

        # self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        key_for_x =  self.key(x)   # (B, T, 16)
        query_for_x = self.query(x) # (B, T, 16)
        wei_for_x=  query_for_x @ key_for_x.transpose(-2, -1) * head_size **-0.5 # (B, T, 16) @ (B, 16, T) => (B, T, T)
        # compute attention scores ("affinities")
        wei_for_x = wei_for_x.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei_for_x = functional.softmax(wei, dim=-1) # (B, T, T)
        wei_for_x = self.dropout(wei)
        # perform the weighted aggregation of the values
        value_for_x = self.value(x) # (B,T,C)
        out = wei_for_x @ value_for_x # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

# Multiple Self Attention Heads in Parallel
class MultiHeadAttention(nn.Module):
    

    def __init__(self, num_heads, head_size):
        super().__init__()

        heads = range(num_heads)

        for idx in range(num_heads):
            heads[idx] = Head(head_size=head_size)

        self.heads = nn.ModuleList(heads)
        self.proj = nn.Linear(num_emb, num_emb)

    def forward(self, x):
        outputs = range(len(self.heads))
        for idx, head in enumerate(self.heads):
            outputs[idx] = head(x)

        # Outputs are concatenated on the embedding dimension or channel dimension
        out = torch.cat(out=outputs, dim=-1)
        return out
    
# A Simple Linear Layer with ReLU
class FeedFoward(nn.Module):

    def __init__(self, num_emb):
        super().__init__()
        self.net = nn.Sequential( 
            nn.Linear(num_emb, 4 * num_emb), 
            nn.ReLU(),
            nn.Linear(4 * num_emb, num_emb),
        )

    def forward(self, x):
        return self.net(x)

# Transformer Block: Communication followed by Computation 
class Block(nn.Module):

    def __init__(self, num_emb, num_heads):
        # num_emb: embedding dimension, num_heads: the number of heads we'd like
        super().__init__()
        head_size = num_emb // num_heads
        self.self_att = MultiHeadAttention(num_heads, head_size)
        self.feed_fwd = FeedFoward(num_emb)

        # Adding Layer Normalization
        self.ln1 = nn.LayerNorm()
        self.ln2 = nn.LayerNorm()


    def forward(self, x):
        x = self.self_att(self.ln1(x))
        x = self.feed_fwd(self.ln2(x))
        return x

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
       
        # Token embedding table is used for token identification encoding
        # Position embedding table is used for token position (in reference to the current context) encoding
        self.token_embedding_table = nn.Embedding(vocab_size, num_emb)
        self.position_embedding_table = nn.Embedding(context, num_emb)

        # Self attention heads (multiple in parallel) are used for communication 
        self.self_att_heads = MultiHeadAttention(4, num_emb // 4)

        # After communication information is acquired, computation takes place using the Feed Forward Layer
        self.feed_forward = FeedFoward(num_emb=num_emb)

        # Language model head used for output
        self.lm_head = nn.Linear(num_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device="cuda")) # (T,C)
        x = tok_emb + pos_emb
        x = self.self_att_heads(x)
        x = self.feed_forward(x) 
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = functional.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -context:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = functional.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
